# "Where did the Village People go?"
## Demonstrator
Here the some of the results from the code will be visulalized on a interactive map via folium.

## Setting Up Workshop

### General Libraries

In [30]:
# Array and Array-Math
import numpy as np
# DataFrames
import pandas as pd
# Geospatial Visualization I
import geopandas as gp
# Geospatial Visualization II
import folium

In [31]:
# This Function Takes the name of a Dataset from Open Data Soft, downloads the GeoJson and turns it into a Geopanda.
def geo_from_ods(title):
    path  = "https://public.opendatasoft.com/explore/dataset/"+title+"/download/?format=geojson"
    geods = gp.read_file(filename = path)
    return geods

## Import and Clean Data

### Geospatial Data

In [32]:
# Geospatial Information
geo_sta_cln = geo_from_ods("bundesland")
geo_dis_cln = geo_from_ods("landkreise-in-germany")

In [33]:
# Find Göttingen 
indx_goe = geo_dis_cln[geo_dis_cln.name_2 == "Göttingen"].index[0]
# Find Osterode am Harz
indx_oah = geo_dis_cln[geo_dis_cln.name_2 == "Osterode am Harz"].index[0]
# Göttingen and Oseterode am Harz the new District Code
geo_dis_cln.loc[indx_goe,"cca_2"] = '03159'
geo_dis_cln.loc[indx_oah,"cca_2"] = '03159'
# Merge Shapes by Code
geo_dis_cln = geo_dis_cln.dissolve(by='cca_2')
geo_dis_cln = geo_dis_cln.reset_index()

In [34]:
# District Level
geo_dis_cln = geo_dis_cln.loc[:,["cca_2", "geometry"]]
geo_dis_cln.columns = ["code","geo"]

### Results from predictive modelling

In [35]:
# Import the chosen results from the Predictive Modeling
data_demo = pd.read_csv(filepath_or_buffer = "data_demo.csv")

In [36]:
# Prepare District Code for Merging
code = np.floor(data_demo.code/1000).astype(int).astype(str)
for idx in code.index:
    if len(code[idx]) == 4:
        code[idx] = "0" + code[idx]
    else:
        pass
code         = pd.DataFrame(code)
data_demo.code = code

### Merge Data

In [37]:
# Merge Districts
data_dis = geo_dis_cln.merge(data_demo, on = "code", how = "inner")
# Merge into One
data = data_dis.append(data_sta, sort = False).reset_index(drop = True)
data = gp.GeoDataFrame(data, geometry = "geo")

### Create HTML Demonstrator

<img src = "./img/Clusters.png" />

In [38]:
# Set Up Folium Map
map_demo = folium.Map(location=[51,9],zoom_start=6)
   
# Define Style Functions, colors as seen above
sf_N = lambda x: {'fillColor': 'blue',
                     'fillOpacity': 0.50,
                     'color': "blue",
                     'opacity': 0.5,
                     'weight' : 2}
sf_M = lambda x: {'fillColor': 'orange',
                     'fillOpacity': 0.50,
                     'color': 'orange',
                     'opacity': 0.5,
                     'weight' : 2}
sf_R = lambda x: {'fillColor': 'green',
                     'fillOpacity': 0.50,
                     'color': 'green',
                     'opacity': 0.5,
                     'weight' : 2}
sf_A = lambda x: {'fillColor': 'red',
                     'fillOpacity': 0.50,
                     'color': 'red',
                     'opacity': 0.5,
                     'weight' : 2}
sf_X = lambda x: {'fillColor': 'black',
                     'fillOpacity': 0.50,
                     'color': 'black',
                     'opacity': 0.5,
                     'weight' : 2}

# Draw Overlays
for idx in data.index:
    if data.mig_profile[idx] == 0:
        layer = folium.GeoJson(data.geo[idx],style_function = sf_N)
        folium.Popup("<table><tr><td><b>"+ data.name[idx]+"</b></td><td>EDU - Neutral</td></tr><tr><td><b>2018</b></td><td>"+ str(int(data.pred_2018[idx]))+"</td></tr><tr><td><b>2022</b></td><td>"+ str(int(data.pred_2022[idx])) + "</td></tr><tr><td><b>....</b></td><td>....</td></tr><tr><td><b>2026</b></td><td>"+ str(int(data.pred_2026[idx]))+"</td></tr></table>").add_to(layer)
        layer.add_to(map_demo)                
    elif data.mig_profile[idx] == 1:
        layer = folium.GeoJson(data.geo[idx],style_function = sf_M)
        folium.Popup("<table><tr><td><b>"+ data.name[idx]+"</b></td><td>EDU - Magnet</td></tr><tr><td><b>2018</b></td><td>"+ str(int(data.pred_2018[idx]))+"</td></tr><tr><td><b>2022</b></td><td>"+ str(int(data.pred_2022[idx])) + "</td></tr><tr><td><b>....</b></td><td>....</td></tr><tr><td><b>2026</b></td><td>"+ str(int(data.pred_2026[idx]))+"</td></tr></table>").add_to(layer)
        layer.add_to(map_demo)  
    elif data.mig_profile[idx] == 2:
        layer = folium.GeoJson(data.geo[idx],style_function = sf_R)
        folium.Popup("<table><tr><td><b>"+ data.name[idx]+"</b></td><td>EDU - Repulsing</td></tr><tr><td><b>2018</b></td><td>"+ str(int(data.pred_2018[idx]))+"</td></tr><tr><td><b>2022</b></td><td>"+ str(int(data.pred_2022[idx])) + "</td></tr><tr><td><b>....</b></td><td>....</td></tr><tr><td><b>2026</b></td><td>"+ str(int(data.pred_2026[idx]))+"</td></tr></table>").add_to(layer)
        layer.add_to(map_demo)
    elif data.mig_profile[idx] == 3:
        layer = folium.GeoJson(data.geo[idx],style_function = sf_A)
        folium.Popup("<table><tr><td><b>"+ data.name[idx]+"</b></td><td>EDU - Attracting</td></tr><tr><td><b>2018</b></td><td>"+ str(int(data.pred_2018[idx]))+"</td></tr><tr><td><b>2022</b></td><td>"+ str(int(data.pred_2022[idx])) + "</td></tr><tr><td><b>....</b></td><td>....</td></tr><tr><td><b>2026</b></td><td>"+ str(int(data.pred_2026[idx]))+"</td></tr></table>").add_to(layer)
        layer.add_to(map_demo)

# Add Overlas to Map
folium.LayerControl().add_to(map_demo)

# Save Map
map_demo.save('vp_demo.html')